In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.utils import to_categorical
import numpy as np
from tensorflow.keras.regularizers import l2

from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

4 Physical GPUs, 1 Logical GPU


In [3]:
s25 = np.load('s25.npy')
h25 = np.load('h25.npy')

In [4]:
s25_labels = np.array([0 for _ in range(0,len(s25))])
h25_labels = np.array([1 for _ in range(0,len(h25))])

In [5]:
X = np.append(s25,h25,axis=0)

In [6]:
y = np.append(s25_labels,h25_labels,axis=0)

In [7]:
X.shape

(1142, 6250, 19)

In [8]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 10, 32
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='selu', input_shape=(n_timesteps,n_features)))
    model.add(Conv1D(filters=64, kernel_size=3,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='selu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation= 'selu' ))
    model.add(Dropout(0.25))
    model.add(Dense(1,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

In [9]:
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print( ' Accuracy: %.3f%% (+/-%.3f) ' % (m, s))

In [10]:
def run_experiment(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    
    testX -= mean
    testX /= std
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model(trainX, trainy, testX, testy)
        score = score * 100.0
        print( ' >#%d: %.3f ' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

In [11]:
run_experiment(X,y)

Epoch 1/10
29/29 [==============================] - 1s 19ms/step - loss: 9.0284 - accuracy: 0.6177
Epoch 2/10
29/29 [==============================] - 1s 18ms/step - loss: 5.4622 - accuracy: 0.8927
Epoch 3/10
29/29 [==============================] - 1s 18ms/step - loss: 3.9144 - accuracy: 0.9704
Epoch 4/10
29/29 [==============================] - 1s 18ms/step - loss: 2.9132 - accuracy: 0.9901
Epoch 5/10
29/29 [==============================] - 1s 17ms/step - loss: 2.3099 - accuracy: 0.9890
Epoch 6/10
29/29 [==============================] - 1s 17ms/step - loss: 1.9452 - accuracy: 0.9945
Epoch 7/10
29/29 [==============================] - 1s 18ms/step - loss: 1.6496 - accuracy: 0.9989
Epoch 8/10
29/29 [==============================] - 1s 18ms/step - loss: 1.5117 - accuracy: 0.9836
Epoch 9/10
29/29 [==============================] - 1s 18ms/step - loss: 1.5010 - accuracy: 0.9858
Epoch 10/10
29/29 [==============================] - 1s 19ms/step - loss: 2.0616 - accuracy: 0.9770
 >#1: 58.

In [12]:
def evaluate_model_2(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 10, 32
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(Conv1D(filters=64,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), kernel_size=3, activation='selu', input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=64,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), kernel_size=3, activation='selu'))
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=64,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), kernel_size=3, activation='selu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation= 'selu' ))
    model.add(Dropout(0.25))
    model.add(Dense(1,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

In [13]:
def run_experiment_2(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    
    testX -= mean
    testX /= std
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model_2(trainX, trainy, testX, testy)
        score = score * 100.0
        print( ' >#%d: %.3f ' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

In [14]:
run_experiment_2(X,y)

Epoch 1/10
29/29 [==============================] - 1s 26ms/step - loss: 28.0266 - accuracy: 0.4896
Epoch 2/10
29/29 [==============================] - 1s 26ms/step - loss: 8.5479 - accuracy: 0.5005
Epoch 3/10
29/29 [==============================] - 1s 26ms/step - loss: 7.4490 - accuracy: 0.5137
Epoch 4/10
29/29 [==============================] - 1s 26ms/step - loss: 6.3691 - accuracy: 0.5126
Epoch 5/10
29/29 [==============================] - 1s 26ms/step - loss: 5.6805 - accuracy: 0.4830
Epoch 6/10
29/29 [==============================] - 1s 26ms/step - loss: 5.0677 - accuracy: 0.4984
Epoch 7/10
29/29 [==============================] - 1s 26ms/step - loss: 4.5121 - accuracy: 0.5268
Epoch 8/10
29/29 [==============================] - 1s 26ms/step - loss: 4.1259 - accuracy: 0.4962
Epoch 9/10
29/29 [==============================] - 1s 26ms/step - loss: 4.0747 - accuracy: 0.4940
Epoch 10/10
29/29 [==============================] - 1s 25ms/step - loss: 4.0919 - accuracy: 0.5170
 >#1: 38

In [15]:
def evaluate_model_3(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 10, 32
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(Conv1D(filters=64,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), kernel_size=3, activation='selu', input_shape=(n_timesteps,n_features)))
    model.add(Conv1D(filters=64,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), kernel_size=3, activation='selu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation= 'selu' ))
    model.add(Dropout(0.25))
    model.add(Dense(50,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation= 'selu' ))
    model.add(Dropout(0.25))
    model.add(Dense(1,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

In [16]:
def run_experiment_3(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    
    testX -= mean
    testX /= std
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model_3(trainX, trainy, testX, testy)
        score = score * 100.0
        print( ' >#%d: %.3f ' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

In [17]:
run_experiment_3(X,y)

Epoch 1/10
29/29 [==============================] - 1s 19ms/step - loss: 9.3583 - accuracy: 0.6407
Epoch 2/10
29/29 [==============================] - 1s 20ms/step - loss: 8.2959 - accuracy: 0.8412
Epoch 3/10
29/29 [==============================] - 1s 20ms/step - loss: 6.9519 - accuracy: 0.9463
Epoch 4/10
29/29 [==============================] - 1s 20ms/step - loss: 5.4737 - accuracy: 0.9671
Epoch 5/10
29/29 [==============================] - 1s 20ms/step - loss: 4.4056 - accuracy: 0.9836
Epoch 6/10
29/29 [==============================] - 1s 20ms/step - loss: 3.5545 - accuracy: 0.9934
Epoch 7/10
29/29 [==============================] - 1s 20ms/step - loss: 2.8028 - accuracy: 0.9945
Epoch 8/10
29/29 [==============================] - 1s 20ms/step - loss: 2.3810 - accuracy: 0.9923
Epoch 9/10
29/29 [==============================] - 1s 20ms/step - loss: 2.2518 - accuracy: 0.9945
Epoch 10/10
29/29 [==============================] - 1s 20ms/step - loss: 2.3714 - accuracy: 0.9825
 >#1: 72.